In [56]:
import joblib
import os
import pandas as pd
import numpy as np
from dateutil import parser

# Importando o dataset

In [14]:
df_final99 = pd.read_csv('df_final_features.csv', sep='|')
df_final99.head()

,Date,Adj Close,Volume,Var%,Fechamento,Noticias_x,Noticias_y,score,Noticias_x.1,neu_rob,...,scored3,scored4,neu_robd1,neu_robd2,neu_robd3,neu_robd4,neg_finbertd1,neg_finbertd2,neg_finbertd3,neg_finbertd4
0,2020-01-09,20.266226,36102700,-0.003279,0,Petrobras refineries show a drop in global oil...,refinarias da petrobras apresentam queda na c...,-3,Petrobras refineries show a drop in global oil...,0.290451,...,0.0,0.0,0.027394,0.081840,0.125774,0.718645,0.000000,0.000000,0.000000,-0.999990
1,2020-01-10,20.179560,25397500,-0.004276,0,petrobras may create subsidiaries to sell 8 re...,petrobras pode criar subsidiárias para vender...,-2,petrobras may create subsidiaries to sell 8 re...,0.061709,...,3.0,0.0,0.290451,0.027394,0.081840,0.125774,-0.999985,0.000000,0.000000,0.000000
2,2020-01-13,20.219559,30676800,0.001982,1,the sale of petrobras refinery in bahia almost...,quase fechada a venda de refinaria da petrob...,7,the sale of petrobras refinery in bahia almost...,0.458953,...,-6.0,3.0,0.061709,0.290451,0.027394,0.081840,0.000000,-0.999985,0.000000,0.000000
3,2020-01-14,19.999565,39796500,-0.010880,0,petrobras fertilizer plant in paraná closes do...,fábrica de fertilizantes da petrobras no para...,-12,petrobras fertilizer plant in paraná closes do...,0.325856,...,-3.0,-6.0,0.458953,0.061709,0.290451,0.027394,0.000000,0.000000,-0.999985,0.000000
4,2020-01-15,19.699570,34405700,-0.015000,0,petrobras sells assets in nigeria and closes a...,petrobras vende ativos na nigéria e encerra a...,-8,petrobras sells assets in nigeria and closes a...,0.507882,...,-2.0,-3.0,0.325856,0.458953,0.061709,0.290451,0.000000,0.000000,0.000000,-0.999985


In [18]:
df_final99['Date'] = pd.to_datetime(df_final99['Date'])

# Treinando até 02-01-2022 e predizendo até 02-02-2022

No primeiro mes a frequencia de retreino foi de 30 dias pq o modelo errou bastante. Na prática no final do mês observariamos a baixa acurácia e seria realizado o retreino mesmo que a politica fosse retreinar a cada 90 dias

In [132]:
#Importando o modelo
ensemblevote = joblib.load(open(os.path.join('modelos/modelo-Ensemble.pkl'),"rb"))

In [133]:
treinar_ate_data_str = '2022-01-02'
data_limite_teste = '2022-02-01'

X_train = df_final99[(df_final99['Date'] <= parser.parse(treinar_ate_data_str))][['score','neu_robd4','neg_finbertd2','scored3']]
y_train = df_final99[(df_final99['Date'] <= parser.parse(treinar_ate_data_str))][['Fechamento']]
y_train = np.ravel(y_train)   
    
X_test = df_final99[(df_final99['Date'] > parser.parse(treinar_ate_data_str)) & (df_final99['Date'] <= parser.parse(data_limite_teste))][['score','neu_robd4','neg_finbertd2','scored3']]
y_test = df_final99[(df_final99['Date'] > parser.parse(treinar_ate_data_str)) & (df_final99['Date'] <= parser.parse(data_limite_teste))][['Fechamento']]
y_test = np.ravel(y_test) 

ensemblevote.fit(X_train,y_train)

#Predizendo y
y_pred = model.predict(X_test)
ensemblevote.score(X_test, y_test)

0.35294117647058826

## Treinando até 02-02-2022 e predizendo até 01-04-2022
Com o modelo treinado em fevereiro foi possivel melhorar a acurácia sobre as predições nos meses 2,3 e 4,5  6

In [134]:
treinar_ate_data_str = '2022-02-02'
data_limite_teste = '2022-04-01'

X_train = df_final99[(df_final99['Date'] <= parser.parse(treinar_ate_data_str))][['score','neu_robd4','neg_finbertd2','scored3']]
y_train = df_final99[(df_final99['Date'] <= parser.parse(treinar_ate_data_str))][['Fechamento']]
y_train = np.ravel(y_train)   
    
X_test = df_final99[(df_final99['Date'] > parser.parse(treinar_ate_data_str)) & (df_final99['Date'] <= parser.parse(data_limite_teste))][['score','neu_robd4','neg_finbertd2','scored3']]
y_test = df_final99[(df_final99['Date'] > parser.parse(treinar_ate_data_str)) & (df_final99['Date'] <= parser.parse(data_limite_teste))][['Fechamento']]
y_test = np.ravel(y_test) 

ensemblevote.fit(X_train,y_train)
quadratic.fit(X_train,y_train)

#Predizendo y
y_pred = model.predict(X_test)

ensemblevote.score(X_test, y_test)

0.6206896551724138

## Treinando até 01-04-2022 e predizendo até 30-06-2022
Reservamos os 3 ultimos meses para avaliar a acurácia, portanto o modelo terá q ser retreinado no dia 01-04-2022

In [135]:
treinar_ate_data_str = '2022-04-01'
data_limite_teste = '2022-06-30'

X_train = df_final99[(df_final99['Date'] <= parser.parse(treinar_ate_data_str))][['score','neu_robd4','neg_finbertd2','scored3']]
y_train = df_final99[(df_final99['Date'] <= parser.parse(treinar_ate_data_str))][['Fechamento']]
y_train = np.ravel(y_train)   
    
X_test = df_final99[(df_final99['Date'] > parser.parse(treinar_ate_data_str)) & (df_final99['Date'] <= parser.parse(data_limite_teste))][['score','neu_robd4','neg_finbertd2','scored3']]
y_test = df_final99[(df_final99['Date'] > parser.parse(treinar_ate_data_str)) & (df_final99['Date'] <= parser.parse(data_limite_teste))][['Fechamento']]
y_test = np.ravel(y_test) 

ensemblevote.fit(X_train,y_train)
quadratic.fit(X_train,y_train)

#Predizendo y
y_pred = model.predict(X_test)

ensemblevote.score(X_test, y_test)

0.5434782608695652